# Body text parsing

In [12]:
import pandas as pd
import os
from tqdm import tqdm
import json
import re
from IPython.display import JSON

import preprocessing
import data_access

In [4]:
# pdf
source_path = 'data/cord-19/document_parses/pdf_json/'
source_filelist = os.listdir(source_path)

dest_path = 'data/cord-19/body_text/pdf/'

for filename in tqdm(source_filelist):
    source = source_path + filename
    dest = dest_path + filename.split('.')[0] + '.txt'
    with open(dest, 'w') as dest_file:
        dest_file.write(data_access.get_body_text_from_path(source))

100%|██████████| 191570/191570 [10:03<00:00, 317.38it/s]


In [5]:
# pmc
source_path = 'data/cord-19/document_parses/pmc_json/'
source_filelist = os.listdir(source_path)

dest_path = 'data/cord-19/body_text/pmc/'

for filename in tqdm(source_filelist):
    source = source_path + filename
    dest = dest_path + filename.split('.')[0] + '.txt'
    with open(dest, 'w') as dest_file:
        dest_file.write(data_access.get_body_text_from_path(source))

100%|██████████| 147047/147047 [08:13<00:00, 297.96it/s]


In [7]:
path = 'data/processed/metadata_clean.csv'
df = preprocessing.load_cleaned_metadata(path)

# delete rows that lack both pdf and pmc paths
c1 = df.pdf_json_files != 'none'
c2 = df.pmc_json_files != 'none'
df = df[c1 ^ c2]

In [8]:
def uids_to_filelist(df):
    
    uids = df.cord_uid.values
    filelist = []
    
    for i in range(len(uids)):
        row = df.iloc[i]
        pdf = row.pdf_json_files
        pmc = row.pmc_json_files
        if pdf != 'none':
            if ';' in pdf:
                file = pdf.split(';')[0].strip()
            else:
                file = pdf
        elif pmc != 'none':
            if ';' in pmc:
                file = pmc.split(';')[0].strip()
            else:
                file = pmc
        filelist.append(file)
    return filelist

In [10]:
filelist = uids_to_filelist(df)
print(len(df.cord_uid.values))
print(len(filelist))

54785
54785


In [30]:
dest_dir = 'data/cord-19/body_text/uid/'
uids = df.cord_uid.values

for uid, file in tqdm(zip(uids, filelist)):
    source = 'data/cord-19/' + file
    dest = dest_dir + uid + '.txt'
    with open(dest, 'w') as file:
        file.write(data_access.get_body_text_from_path(source))

54785it [03:18, 276.55it/s]
